In [1]:
from PIL import Image
from numpy import array
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras import regularizers

Using TensorFlow backend.


In [5]:
data = np.load("zdmdtImages.npz")
print data['Y'][:-3].shape,data['X'].shape

batch_size = 32
num_classes = 2
epochs = 12
# splitting data into train and test
x_train, x_test, y_train, y_test= train_test_split(data['X'], data['Y'][:-3], test_size=0.20, random_state=42, stratify=data['Y'][:-3])
# splitting train into valid dataset
print x_train.shape
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.30, random_state=42, stratify=y_train)
print x_train.shape
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

print('x_train shape: ', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

(35425,) (35425, 22, 24, 4)
(28340, 22, 24, 4)
(19838, 22, 24, 4)
('x_train shape: ', (19838, 22, 24, 4))
(19838, 'train samples')
(7085, 'test samples')


In [6]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
y_valid = keras.utils.to_categorical(y_valid, num_classes)

In [14]:
# with regularisation in dense layer
model = Sequential()
model.add(Conv2D(8, kernel_size=(3, 3),
                 strides=(2,2),
                 activation='relu',
                 input_shape=(22, 24, 4)))
model.add(Conv2D(16, (5, 5), strides=(2,2), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), padding="same"))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
model.summary()
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adagrad(),
              metrics=['accuracy'])
# adaptive gradient, batch size should increase, avoid overfitting, increase epochs

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 10, 11, 8)         296       
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 3, 4, 16)          3216      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 2, 2, 16)          0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 2, 2, 16)          0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 64)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 128)               8320      
_________________________________________________________________
dropout_8 (Dropout)          (None, 128)               0         
__________

In [15]:
earlyStopping = keras.callbacks.EarlyStopping(monitor='val_loss',min_delta=1e-6,patience=5,verbose=0, mode='auto')

model.fit(x_train, y_train,
          batch_size=64,
          epochs=40,
          verbose=1,
          validation_data=(x_valid,y_valid),
          callbacks=[earlyStopping])

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 19838 samples, validate on 8502 samples
Epoch 1/40
19838/19838 [==============================] - 10s 510us/step - loss: 0.4919 - acc: 0.8673 - val_loss: 2.1514 - val_acc: 0.8679
Epoch 2/40
19838/19838 [==============================] - 9s 477us/step - loss: 0.4064 - acc: 0.8679 - val_loss: 2.1410 - val_acc: 0.8679
Epoch 3/40
19838/19838 [==============================] - 11s 546us/step - loss: 0.3736 - acc: 0.8679 - val_loss: 1.7001 - val_acc: 0.8792
Epoch 4/40
19838/19838 [==============================] - 12s 617us/step - loss: 0.3539 - acc: 0.8679 - val_loss: 1.6339 - val_acc: 0.8749
Epoch 5/40
19838/19838 [==============================] - 11s 550us/step - loss: 0.3428 - acc: 0.8679 - val_loss: 1.2987 - val_acc: 0.8677
Epoch 6/40
19838/19838 [==============================] - 10s 519us/step - loss: 0.3392 - acc: 0.8679 - val_loss: 1.4143 - val_acc: 0.8681
Epoch 7/40
19838/19838 [==============================] - 12s 599us/step - loss: 0.3349 - acc: 0.8679 - val_loss: 1.34

In [16]:
# without regularisation 
model = Sequential()
model.add(Conv2D(8, kernel_size=(3, 3),
                 strides=(2,2),
                 activation='relu',
                 input_shape=(22, 24, 4)))
model.add(Conv2D(16, (5, 5), strides=(2,2), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), padding="same"))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
model.summary()
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adagrad(),
              metrics=['accuracy'])

earlyStopping = keras.callbacks.EarlyStopping(monitor='val_loss',min_delta=1e-6,patience=5,verbose=0, mode='auto')

model.fit(x_train, y_train,
          batch_size=64,
          epochs=40,
          verbose=1,
          validation_data=(x_valid,y_valid),
          callbacks=[earlyStopping])

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 10, 11, 8)         296       
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 3, 4, 16)          3216      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 2, 2, 16)          0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 2, 2, 16)          0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 64)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 128)               8320      
_________________________________________________________________
dropout_10 (Dropout)         (None, 128)               0         
__________

In [8]:
#2nd attempt at CNN close to paper implementation
model = Sequential()
model.add(Conv2D(64, kernel_size=(3, 3),
                 activation = 'relu',
                 input_shape = (22, 24, 4)))
model.add(MaxPooling2D(pool_size=(2, 2), padding="same"))
model.add(Dropout(0.1))
model.add(Conv2D(128,kernel_size=(3, 3), activation = 'relu'))
model.add(Conv2D(256,kernel_size=(3, 3), activation = 'relu'))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))
model.summary()
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adagrad(),
              metrics=['accuracy'])
earlyStopping = keras.callbacks.EarlyStopping(monitor='val_loss',min_delta=1e-6,patience=3,verbose=0, mode='auto')

model.fit(x_train, y_train,
          batch_size=64,
          epochs=40,
          verbose=1,
          validation_data=(x_valid,y_valid),
          callbacks=[earlyStopping])

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 20, 22, 64)        2368      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 10, 11, 64)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 10, 11, 64)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 8, 9, 128)         73856     
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 6, 7, 256)         295168    
_________________________________________________________________
flatten_3 (Flatten)          (None, 10752)             0         
_________________________________________________________________
dense_7 (Dense)              (None, 512)               5505536   
__________

In [18]:
ypred = model.predict(x_test)

In [20]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test.argmax(axis=1),ypred.argmax(axis=1))

array([[6149,    0],
       [ 936,    0]])